## Matching Type modèle 

L’objectif est de trouver une méthode permettant de faire correspondre les types de véhicules présents sur [EV Database](https://ev-database.org/) avec ceux issus du scraping et ceux retournés par les API, pour lesquels les types sont parfois erronés ou incohérents.

Un exemple de véhicules scrappés est disponible dans la feuille Google Sheet [Courbes OS](https://docs.google.com/spreadsheets/d/1TBHjDtGkqcWgd4TcYBgzR4b6Yn5OV_hFfyx3Wty7kW8/edit?gid=0#gid=0).

Méthode actuellement utilisée :
- Masquage sur le nom du constructeur (OEM)
- Masquage sur le nom du modèle
- Si la capacité batterie est disponible → masquage également sur ce critère
- Recherche de la ressemblance maximale entre le type du véhicule et ceux présents dans DBeaver en utilisant rapidfuzz



In [ ]:
from core.gsheet_utils import *
from core.sql_utils import *
from rapidfuzz import process, fuzz


## Load data 

In [ ]:
# load database from dbeaver
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT vm.model_name, vm.id, vm.type, o.oem_name, b.capacity FROM vehicle_model vm
                                  join OEM o on vm.oem_id=o.id
                                  join battery b on b.id=vm.battery_id;"""), con)


In [ ]:
# Load database from courbes OS 
df = load_excel_data(get_gspread_client(), "202505 - Courbes SoH", "Courbes OS")
df_sheet = pd.DataFrame(columns=df[0,:8], data=df[1:,:8])




In [ ]:
def find_db_type(row, db_df):
    """_summary_

    Args:
        row (pd.Series): 
        db_df (pd.DataFrame): dataframe avec les colonnes model_name, id, type, oem_name, capacity

    Returns:
        uuid.UUID: id du modèle
    """

    #On récupère les infos 
    oem = row['OEM'].lower()
    model_target = row['Modèle'].lower()
    version_target = row['Type'].lower()
    # filtre sur l'oem 
    subset = db_df[db_df['oem_name'] == oem].copy()
    
    # Trouver la meilleure correspondance
    match_model = process.extractOne(model_target, subset['model_name'], scorer=fuzz.token_sort_ratio)
    if match_model :
        match_model_name, score, index = match_model
        # filtre sur le nom du modèle
        subset = subset[subset['model_name']==match_model_name]
        # on cherche la batetrie qui avec la capacité la + proche
        if row['battery_capacity'] != 'unknown':
            battery_target = float(row['battery_capacity'].replace('kWh', '').replace('kwh', '').strip())
            subset["distance"] = (subset["capacity"] - battery_target).abs()
            min_distance = subset["distance"].min()
            closest_rows = subset[subset["distance"] == min_distance]
            # Si +sieurs batterie -> type le plus ressemblant
            match_type = process.extractOne(version_target, closest_rows['type'], scorer=fuzz.token_sort_ratio)
            match_model_type, score, index = match_type
            return closest_rows.loc[index, "type"]

        else:  
            if subset['type'] is None:
                 return None
            
            # type le plus ressemblant
            match_type = process.extractOne(version_target, subset['type'], scorer=fuzz.token_sort_ratio)
            match_model_type, score, index = match_type
            
            return subset.loc[index, "type"]

        

In [ ]:
df_sheet['model_id'] = df_sheet.apply(lambda row: find_db_type(row, dbeaver_df), axis=1)


In [ ]:
df_sheet